In [1]:
from mlflow import version
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking import MlflowClient, client
from pprint import pprint
import joblib
import os

def log_production_model(config_path):
    config = read_params(config_path)

    mlflow_config = config["mlflow_config"]
    model_name = mlflow_config["registered_model_name"]
    remote_server_uri = mlflow_config["remote_server_uri"]
    
    mlflow.set_tracking_uri(remote_server_uri)

    runs = mlflow.search_runs(experiment_ids=1)
    print(runs.head())
    lowest = runs["metrics.mae"].sort_values(ascending=True)[0]
    lowest_run_id = runs[runs["metrics.mae"]==lowest]["run_id"][0]

    client = MlflowClient()
    for mv in client.search_model_versions(f"name='{model_name}'"):
        mv = dict(mv)

        if mv["run_id"] == lowest_run_id:
            current_version = mv["version"]
            logged_model = mv["source"]
            pprint(mv, indent=4)
            client.transition_model_version_stage(
                name=model_name,
                version=current_version,
                stage='Production'
            )
        else:
            current_version = mv["version"]
            client.transition_model_version_stage(
                name=model_name,
                version=current_version,
                stage='Staging'
            )

In [2]:
pwd

'/Users/sanket/Personal/Github/WineProject/notebooks'

In [3]:
BASE_DIR = os.path.realpath('.')
BASE_DIR

'/Users/sanket/Personal/Github/WineProject/notebooks'

In [4]:
cd ..

/Users/sanket/Personal/Github/WineProject


In [5]:
!ls

Procfile           dvc.lock           report             template.py
README.md          dvc.yaml           requirements.txt   tests
app.py             mlflow.db          saved_models       tox.ini
artifacts          notebooks          setup.py           webapp
data               params.yaml        src
data_given         prediction_service src.egg-info


In [6]:
log_production_model("params.yaml")

                             run_id experiment_id    status  \
0  0fae84374df049f1a362d11f32eb24a2             1  FINISHED   
1  45456492f4fa43f8a58a5c4afc394f0e             1  FINISHED   
2  a7bfc23b966f429a9b6a73cd0d5bad19             1  FINISHED   

                                        artifact_uri  \
0  ./artifacts/1/0fae84374df049f1a362d11f32eb24a2...   
1  ./artifacts/1/45456492f4fa43f8a58a5c4afc394f0e...   
2  ./artifacts/1/a7bfc23b966f429a9b6a73cd0d5bad19...   

                        start_time                         end_time  \
0 2021-07-14 05:15:22.753000+00:00 2021-07-14 05:15:23.133000+00:00   
1 2021-07-14 05:14:46.831000+00:00 2021-07-14 05:14:47.858000+00:00   
2 2021-07-13 20:01:23.450000+00:00 2021-07-13 20:01:24.247000+00:00   

   metrics.mae  metrics.rmse  metrics.r2 params.l1_ratio params.alpha  \
0     0.655146      0.803119    0.013015             0.4          0.9   
1     0.659818      0.805002    0.008382            0.89         0.88   
2     0.618863    